# Visualize TimePix3 data

read from "hdf5" files created by Kuepper et al. (Hubertus B.)

### Imports

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

# plt.rc('figure', figsize=(20, 10))


### Define file

In [3]:
hdf_file_name = '../data/20191031-052753_CH3I_000000.hdf5'

### Open HDF file

In [4]:
h_file = h5py.File(hdf_file_name,'r')

### Read from HDF file

In [5]:
tof = h_file['centroided/tof'][:]
x_pos = h_file['centroided/x'][:]
y_pos = h_file['centroided/y'][:]

### Utility function

In [6]:
def generate_plots(tof, x_pos, y_pos, hist_bins='auto'):
    
    fig = plt.figure()
    plt.hist(tof, bins= hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [s]')
    plt.ylabel('number of events')
       
    fig = plt.figure()
    plt.plot(tof, x_pos,'.')
    plt.title("ToF vs x_pos")
    plt.xlabel('ToF [s]')
    plt.ylabel('x_pos [px]')

    fig = plt.figure()
    plt.plot(tof, y_pos,'.')
    plt.title("ToF vs y_pos")
    plt.xlabel('ToF [s]')
    plt.ylabel('y_pos [px]')

    plt.show()

def generate_2d_hists(tof, x_pos, y_pos ,bin_size = 1000):

    fig = plt.figure()
    plt.hist2d(tof, x_pos, bins= (bin_size,bin_size))
    plt.title('histogram: time-of-flight / x_pos')
    plt.xlabel('ToF [s]')
    plt.ylabel('x_pos')

    fig = plt.figure()
    plt.hist2d(tof, y_pos, bins= (bin_size,bin_size))
    plt.title('histogram: time-of-flight / y_pos')
    plt.xlabel('ToF [s]')
    plt.ylabel('y_pos')
    
    plt.show()
    
    
def cut_in_tof_from_back(tof, x_pos, y_pos, cutoff):
    cutted_x_pos = x_pos[tof < cutoff]
    cutted_y_pos = y_pos[tof < cutoff]
    cutted_tof = tof[tof < cutoff]
    return cutted_tof, cutted_x_pos, cutted_y_pos


In [7]:
generate_plots(tof, x_pos, y_pos, hist_bins = 1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Reduce data to one single FEL train

In [8]:
cutoff = 12E-6
tof, x_pos, y_pos =  cut_in_tof_from_back(tof, x_pos, y_pos, cutoff)

generate_plots(tof, x_pos, y_pos, hist_bins = 1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
generate_2d_hists(tof, x_pos, y_pos)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …